In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(columns = (['EIN', 'NAME']))
dropped_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
dropped_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Bin APPLICATION_TYPE and CLASSIFICATION value counts
application_df_counts = dropped_df['APPLICATION_TYPE'].value_counts()
print(application_df_counts)
classification_df = dropped_df['CLASSIFICATION'].value_counts()
print(classification_df)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# Use a cutoff value of 50 for both features, and create a list of application types to be replaced

cutoff_value = 50

application_types_to_replace = application_df_counts[application_df_counts <= cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
dropped_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Use a cutoff value of 50 for both features, and create a list of classifications to be replaced

cutoff_value = 50

classifications_to_replace = classification_df[classification_df <= cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
  dropped_df['CLASSIFICATION'] = dropped_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
binned_df = dropped_df['CLASSIFICATION'].value_counts()
binned_df

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      441
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
Name: CLASSIFICATION, dtype: int64

In [9]:
copy_df = dropped_df.copy()
copy_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummy_df = pd.get_dummies(copy_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']])
application_dummy_df.sample(10)

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
11509,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
19289,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
27503,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
14644,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
15260,1,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
5324,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
22580,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
13772,1,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4709,1,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
27904,1,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_dummy_df.drop(columns = ['IS_SUCCESSFUL'])

y = application_dummy_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
print(X_train.shape)

(25724, 44)


In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = 44

model_1 = tf.keras.models.Sequential()

# First hidden layer
model_1.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim = num_features))

# Second hidden layer
model_1.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
model_1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                900       
                                                                 
 dense_7 (Dense)             (None, 15)                315       
                                                                 
 dense_8 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1,231
Trainable params: 1,231
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
model_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
model_1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5866 - accuracy: 0.7156
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5632 - accuracy: 0.7227
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5595 - accuracy: 0.7240
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5581 - accuracy: 0.7243
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5575 - accuracy: 0.7241
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7248
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7257
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5564 - accuracy: 0.7257
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.7261
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = model_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7284 - 311ms/epoch - 1ms/step
Loss: 0.558331310749054, Accuracy: 0.728396475315094


In [25]:
# Export our model to HDF5 file
model_1.save("AlphabetSoupCharity_Optimization_2.h5")